Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [4]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
import gurobipy as gp
from gurobipy import GRB
import math


In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


#### Fitness function

In [3]:
def fitness(state, instance):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [un_sparse(instance, i) for i, t in enumerate(state) if t],
            np.array([False for _ in range(instance.shape[1])]),
        )
    )
    
    return valid, -cost

def un_sparse(instance, i):
    a = []
    num_points = instance.shape[1]

    for j in range(num_points):
        a.append(instance[i,j])

    return np.array(a)

#### Generate instances

In [7]:
x_100_3 = make_set_covering_problem(100, 100, 0.3) #solution 5
x_100_7 = make_set_covering_problem(100, 100, 0.7)
x_1000_3 = make_set_covering_problem(1000, 1000, 0.3) #solution 10
x_1000_7 = make_set_covering_problem(1000, 1000, 0.7)
x_5000_3 = make_set_covering_problem(5000, 5000, 0.3)
x_5000_7 = make_set_covering_problem(5000, 5000, 0.3)

## Hill Climbing

In [5]:
def hill_climbing(instance, init = None, num_flip = 1):
    if init is None:
        current_state = [choice([False]) for _ in range(instance.shape[1])]
    else:
        current_state = init
    num_points = instance.shape[1]

    current_covered = 0
    current_cost = -math.inf
    num_call = 0

    for _ in range(1000):
        new_state = tweak(current_state, instance,  num_flip)
        #print(f'new state: {fitness(new_state, instance)}')
        new_covered, new_cost = fitness(new_state, instance)
        num_call = num_call + 1

        if  new_covered > current_covered:
            current_state = new_state
            current_covered = new_covered
            current_cost = new_cost
            best_num_call = num_call
        else:
            if (new_covered == current_covered) and (new_cost > current_cost):
                current_state = new_state
                current_covered = new_covered
                current_cost = new_cost
                best_num_call = num_call


    
    
    return (current_covered, current_cost, best_num_call)

def tweak(state, instance, num_flip = 1):
    new_state = copy(state)
    indices = [randint(0, instance.shape[1] - 1) for _ in range(num_flip)]

    for i in indices:
        new_state[i] = not new_state[i]
    return new_state

## Integer Programming

In [28]:
def integer_programming(instance):
    var_len = instance.shape[0]
    constr_len = instance.shape[1]
    model = gp.Model('Set Covering')
    model.setParam('TimeLimit', 5*60)

    X = model.addMVar(var_len, vtype=GRB.BINARY, name = 'X')
    A = np.array(instance.toarray(), dtype = int).T

    model.setObjective(X.sum(), GRB.MINIMIZE)
    model.addConstr( A @ X >= np.ones(constr_len))

    model.update()
    model.optimize()

    X_sol = []
    for v in model.getVars():
        X_sol.append(v.X)

    return sum(X_sol)

In [12]:
result_100_3 = integer_programming(x_100_3)
result_1000_3 = integer_programming(x_1000_3)
result_5000_3 = integer_programming(x_5000_3)
result_100_7 = integer_programming(x_100_7)
result_1000_7 = integer_programming(x_1000_7)
result_5000_7 = integer_programming(x_5000_7)

print(f'integer programming')
print(f'size 100 density 0.3 : {result_100_3}')
print(f'size 1000 density 0.3 : {result_1000_3}')
print(f'size 5000 density 0.3 : {result_5000_3}')
print(f'size 100 density 0.7 : {result_100_7}')
print(f'size 1000 density 0.7 : {result_1000_7}')
print(f'size 5000 density 0.7 : {result_5000_7}')

Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 100 columns and 3066 nonzeros
Model fingerprint: 0x65667521
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 9.0000000
Presolve time: 0.00s
Presolved: 100 rows, 100 columns, 3066 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 3.329505e+00, 180 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.32951  

In [8]:
print(f'hill climbing')
print(f'size 100 density 0.3 : {hill_climbing(x_100_3)}')
print(f'size 1000 density 0.3 : {hill_climbing(x_1000_3)}')
print(f'size 5000 density 0.3 : {hill_climbing(x_5000_3)}')
# (coverd, cost, number of evaluation)

hill climbing
size 100 density 0.3 : (100, -10, 13)
size 1000 density 0.3 : (1000, -14, 788)
size 5000 density 0.3 : (5000, -20, 597)


## Simulated Annealing